In [2]:
from zipfile import ZipFile

kmz = ZipFile('Functional Pumps.kmz', 'r')
kml = kmz.open('doc.kml', 'r').read()

In [ ]:
from lxml import html

doc = html.fromstring(kml)

In [3]:
for pm in doc.cssselect('Document Placemark'):
    tmp = pm.cssselect('track')
    name = pm.cssselect('name')[0].text_content()
    if len(tmp):
        # Track Placemark
        tmp = tmp[0]  # always one element by definition
        for desc in tmp.iterdescendants():
            content = desc.text_content()
            if desc.tag == 'coord':
                print(content)
            else:
                print("Skipping empty tag %s" % desc.tag)
    else:
        # Reference point Placemark
        coord = pm.cssselect('Point coordinates')[0].text_content()
        print(coord)


          17.640604,4.193693,0
        

          16.532527,5.887524,0
        

          16.528749,5.882192,0
        

          16.51439,5.839985,0
        

          16.670921,5.639984,0
        

          16.703602,5.561064,0
        

          16.686225,5.593668,0
        

          16.710515,5.557414,0
        

          16.67295,5.635783,0
        

          16.79602,5.475162,0
        

          16.894245,5.425649,0
        

          16.791345,5.48015,0
        

          16.763239,5.514794,0
        

          18.559,4.656166,0
        

          17.442239,6.063913,0
        

          17.450946,6.463887,0
        

          17.435517,6.411522,0
        

          17.457328,6.507838,0
        

          17.134081,6.488481,0
        

          17.334705,6.889235,0
        

          17.313201,6.922389,0
        

          17.312312,6.922892,0
        

          17.277383,6.98921,0
        

          18.27393,6.516155,0
        

          18.271574,6.50

In [14]:
from geopy.distance import distance

def parseKMZFile(kmz_file_path: str) -> list:
    """
    Parses Google My Maps KMZ file into a list of pointmarks.
    
    Parameters
    ---
    - `kmz_file_path` - path to KMZ file to be analyzed.

    Returns
    ---
    A list of placemark coordinates in `[latitude, longitude]` format
    """

    kmz = ZipFile('Functional Pumps.kmz', 'r')
    kml = kmz.open('doc.kml', 'r').read()
    doc = html.fromstring(kml)

    wells_coordinates = []

    for placemark_coordinate in doc.cssselect("Document Placemark Point"):
        well_coordinates_raw: str = placemark_coordinate.text_content()
        well_coordinates_split = well_coordinates_raw.split(',')
        well_coordinates = (
        float(well_coordinates_split[1].strip()),
        float(well_coordinates_split[0].strip())
    )
        wells_coordinates.append(well_coordinates)
    return wells_coordinates

def closestWellFromLocation(origin_coordinates: float, wells_coordinates: list) -> tuple:
    """
    Returns the position of the well that is closest to origin point and distance between them.

    All coordinates should be in `[latitude, longitude]` format.
    """
    distances_to_wells = [
        distance(origin_coordinates, well_coordinates).m for well_coordinates in wells_coordinates
    ]
    minimal_distance = min(distances_to_wells)
    index_of_the_closest_well = distances_to_wells.index(minimal_distance)
    return(
        wells_coordinates[index_of_the_closest_well],
        minimal_distance
    )

In [ ]:
random_point_near_wells = (5.87667, 19.11722)